# 2.06: Policy Simulation — Resource Allocation for Population Health

**Learning Objectives:**
- Apply cost-effectiveness analysis to real-world resource allocation decisions
- Understand diminishing returns and the cost-effectiveness frontier
- Explore the tension between efficiency and equity in health policy
- Critically evaluate the limitations of purely economic approaches to priority-setting

---

## The Challenge

You are advising a regional health authority with a **£50 million budget** for preventive health programmes over the next 5 years. Your task is to allocate this budget across available interventions to maximise population health — measured in DALYs averted.

But maximising DALYs isn't the only consideration:
- **Equity**: Do benefits reach those in greatest need?
- **Uncertainty**: How confident are we in these estimates?
- **Time horizon**: Should we value immediate gains over long-term prevention?

---

## 1. Setup

In [ ]:
# ============================================================
# Bootstrap cell (works both locally and in Colab)
#
# What this cell does:
# - Ensures that we are inside the course repository.
# - In Colab: clones the repository from GitHub if necessary.
# - Loads the course utility module (epi_utils.py).
#
# Important:
# - You may see messages printed below (e.g. from pip or git).
# - Warnings (often in yellow) are usually harmless.
# - If you see a red error traceback, re-run this cell first.
# ============================================================

import os
import sys
import pathlib
import subprocess

# ------------------------------------------------------------
# Configuration: repository location and URL
# ------------------------------------------------------------
REPO_URL = "https://github.com/ggkuhnle/fb2nep-epi.git"
REPO_DIR = "fb2nep-epi"

# ------------------------------------------------------------
# 1. Ensure we are inside the repository
# ------------------------------------------------------------
cwd = pathlib.Path.cwd()

# Case A: we are already in the repository (scripts/epi_utils.py exists)
if (cwd / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd
# Case B: we are in a subdirectory of the repository
elif (cwd.parent / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd.parent
# Case C: we are outside the repository (e.g. in Colab)
else:
    repo_root = cwd / REPO_DIR

    # Clone the repository if not present
    if not repo_root.is_dir():
        print(f"Cloning repository from {REPO_URL} into {repo_root} ...")
        subprocess.run(["git", "clone", REPO_URL, str(repo_root)], check=True)
    else:
        print(f"Using existing repository at {repo_root}")

    # Change working directory to repository root
    os.chdir(repo_root)
    repo_root = pathlib.Path.cwd()

# Add scripts directory to Python path
scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

print(f"Repository root: {repo_root}")
print("Bootstrap completed successfully.")

In [ ]:
# ------------------------------------------------------------
# Import libraries and course utilities
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, VBox, HBox, Output, HTML
from IPython.display import display, clear_output

# Import course utilities from the repository
from epi_utils import (
    INTERVENTIONS, STRATEGIES,
    calculate_dalys_averted, calculate_marginal_cost_per_daly,
    calculate_equity_dalys, evaluate_strategy,
    evaluate_strategy_with_uncertainty, evaluate_with_equity_weights,
    get_intervention_summary, print_strategy_comparison
)

np.random.seed(42)

print("Libraries loaded successfully.")

## 2. The Interventions

Below are 8 interventions you can fund. Cost-effectiveness estimates are derived from published literature, NICE guidance, and PHE analyses.

In [ ]:
# View intervention summary
print("Available Interventions")
print("=" * 100)
display(get_intervention_summary())

In [ ]:
# Display detailed descriptions
print("\nIntervention Details")
print("=" * 100)
for key, v in INTERVENTIONS.items():
    print(f"\n{v['name'].upper()}")
    print(f"  {v['description']}")
    print(f"  → Equity profile: Q1 (most deprived) receives {v['equity_distribution'][0]*100:.0f}% of benefits")

## 3. Diminishing Returns

Real interventions don't scale linearly. The first £1M reaches the most accessible population. Subsequent spending encounters harder-to-reach populations and saturation effects.

In [ ]:
# Visualise diminishing returns
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, (key, intervention) in enumerate(INTERVENTIONS.items()):
    ax = axes[idx]
    spends = np.linspace(0.1, intervention['max_capacity_millions'], 50)
    marginal_costs = [calculate_marginal_cost_per_daly(s, intervention) for s in spends]
    
    ax.plot(spends, marginal_costs, 'b-', linewidth=2)
    ax.axhline(y=20000, color='r', linestyle='--', alpha=0.7, label='NICE threshold')
    ax.set_xlabel('Spend (£M)')
    ax.set_ylabel('Marginal £/DALY')
    ax.set_title(intervention['name'], fontsize=10)
    ax.set_ylim(0, min(50000, max(marginal_costs) * 1.1))

plt.suptitle('Marginal Cost-Effectiveness by Spending Level', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. The Budget Allocation Game

You have **£50 million** to allocate. Use the sliders to set your budget for each intervention.

In [ ]:
TOTAL_BUDGET = 50

# Create sliders
sliders = {}
slider_widgets = []

for key, intervention in INTERVENTIONS.items():
    slider = IntSlider(
        value=0, min=0, max=intervention['max_capacity_millions'], step=1,
        description='', continuous_update=True,
        layout=widgets.Layout(width='250px')
    )
    sliders[key] = slider
    label = HTML(
        value=f"<b>{intervention['name']}</b><br><small>Max: £{intervention['max_capacity_millions']}M</small>",
        layout=widgets.Layout(width='220px')
    )
    slider_widgets.append(HBox([label, slider, HTML(value="£M")]))

budget_html = HTML(value="")
results_output = Output()
charts_output = Output()

def update_dashboard(change=None):
    allocations = {key: slider.value for key, slider in sliders.items()}
    total_spend = sum(allocations.values())
    remaining = TOTAL_BUDGET - total_spend
    
    # Budget display
    if remaining < 0:
        budget_html.value = f"<div style='padding:10px;background:#ffcccc;border-radius:5px;'><h3 style='color:red;margin:0;'>⚠️ OVER BUDGET by £{-remaining}M</h3></div>"
    elif remaining == 0:
        budget_html.value = f"<div style='padding:10px;background:#ccffcc;border-radius:5px;'><h3 style='color:green;margin:0;'>✓ Budget Fully Allocated</h3></div>"
    else:
        budget_html.value = f"<div style='padding:10px;background:#ffffcc;border-radius:5px;'><h3 style='margin:0;'>Budget Remaining: £{remaining}M</h3></div>"
    
    # Calculate results
    results = []
    equity_totals = {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0}
    
    for key, spend in allocations.items():
        intervention = INTERVENTIONS[key]
        dalys = calculate_dalys_averted(spend, intervention)
        equity_dalys = calculate_equity_dalys(dalys, intervention['equity_distribution'])
        for q, val in equity_dalys.items():
            equity_totals[q] += val
        
        if spend > 0:
            avg_cost = (spend * 1_000_000) / dalys if dalys > 0 else 0
            marginal_cost = calculate_marginal_cost_per_daly(spend, intervention)
        else:
            avg_cost, marginal_cost = 0, intervention['base_cost_per_daly']
        
        results.append({
            'intervention': intervention['name'], 'spend': spend, 'dalys': dalys,
            'avg_cost': avg_cost, 'marginal_cost': marginal_cost,
            'time_to_impact': intervention['time_to_impact_years']
        })
    
    total_dalys = sum(r['dalys'] for r in results)
    
    # Results display
    with results_output:
        clear_output(wait=True)
        print("=" * 80)
        print(f"TOTAL DALYs AVERTED: {total_dalys:,.0f}")
        if total_spend > 0:
            print(f"OVERALL COST-EFFECTIVENESS: £{(total_spend * 1_000_000) / total_dalys:,.0f} per DALY")
        print("=" * 80)
        
        results_df = pd.DataFrame(results)
        results_df = results_df[results_df['spend'] > 0].sort_values('dalys', ascending=False)
        if len(results_df) > 0:
            display_df = results_df[['intervention', 'spend', 'dalys', 'avg_cost', 'marginal_cost']].copy()
            display_df.columns = ['Intervention', 'Spend (£M)', 'DALYs', 'Avg £/DALY', 'Marginal £/DALY']
            display(display_df.style.format({
                'Spend (£M)': '£{:.0f}M', 'DALYs': '{:,.0f}',
                'Avg £/DALY': '£{:,.0f}', 'Marginal £/DALY': '£{:,.0f}'
            }).hide(axis='index'))
    
    # Charts
    with charts_output:
        clear_output(wait=True)
        if total_dalys > 0:
            fig = make_subplots(rows=1, cols=2,
                subplot_titles=('DALYs by Intervention', 'Equity: DALYs by Deprivation Quintile'))
            
            funded = sorted([r for r in results if r['spend'] > 0], key=lambda x: x['dalys'], reverse=True)
            if funded:
                fig.add_trace(go.Bar(
                    x=[r['intervention'] for r in funded],
                    y=[r['dalys'] for r in funded],
                    marker_color='steelblue'
                ), row=1, col=1)
            
            quintile_labels = ['Q1 (Most Deprived)', 'Q2', 'Q3', 'Q4', 'Q5 (Least Deprived)']
            quintile_values = [equity_totals[f'Q{i+1}'] for i in range(5)]
            colors = ['#d73027', '#fc8d59', '#fee090', '#91bfdb', '#4575b4']
            fig.add_trace(go.Bar(x=quintile_labels, y=quintile_values, marker_color=colors), row=1, col=2)
            
            fig.update_layout(height=400, showlegend=False)
            fig.update_xaxes(tickangle=45, row=1, col=1)
            fig.show()
            
            q1_share = equity_totals['Q1'] / total_dalys * 100
            print(f"\nEquity: {q1_share:.1f}% of benefits go to most deprived quintile (Q1)")
            if q1_share > 22:
                print("→ Your allocation is PRO-EQUITY")
            elif q1_share < 18:
                print("→ Your allocation may WIDEN INEQUALITIES")

for slider in sliders.values():
    slider.observe(update_dashboard, names='value')

print("BUDGET ALLOCATION GAME - Allocate your £50M:")
display(VBox([budget_html, VBox(slider_widgets), results_output, charts_output]))
update_dashboard()

## 5. Strategy Comparison

Let's compare three pre-defined strategies.

In [ ]:
print_strategy_comparison()

In [ ]:
# Visualise comparison
comparison_results = []
for name, strategy in STRATEGIES.items():
    metrics = evaluate_strategy(strategy['allocations'])
    comparison_results.append({
        'Strategy': name,
        'Total DALYs': metrics['total_dalys'],
        'Q1 Share (%)': metrics['q1_share']
    })

comparison_df = pd.DataFrame(comparison_results)

fig = make_subplots(rows=1, cols=2,
    subplot_titles=('Total DALYs Averted', 'Share to Most Deprived (Q1)'))

colors = ['#2ecc71', '#3498db', '#9b59b6']
fig.add_trace(go.Bar(x=comparison_df['Strategy'], y=comparison_df['Total DALYs'], marker_color=colors), row=1, col=1)
fig.add_trace(go.Bar(x=comparison_df['Strategy'], y=comparison_df['Q1 Share (%)'], marker_color=colors), row=1, col=2)
fig.add_hline(y=20, line_dash="dash", line_color="red", annotation_text="Equal (20%)", row=1, col=2)
fig.update_layout(height=400, showlegend=False, title_text="Strategy Comparison")
fig.show()

## 6. The Efficiency-Equity Frontier

In [ ]:
# Generate random allocations to map the frontier
n_simulations = 500
simulation_results = []

for _ in range(n_simulations):
    raw_weights = np.random.dirichlet(np.ones(8)) * 50
    allocations = {}
    for i, key in enumerate(INTERVENTIONS.keys()):
        allocations[key] = min(raw_weights[i], INTERVENTIONS[key]['max_capacity_millions'])
    
    total = sum(allocations.values())
    if total > 0:
        scale = 50 / total
        allocations = {k: min(v * scale, INTERVENTIONS[k]['max_capacity_millions']) for k, v in allocations.items()}
    
    metrics = evaluate_strategy(allocations)
    simulation_results.append({'dalys': metrics['total_dalys'], 'q1_share': metrics['q1_share']})

# Add named strategies
for name, strategy in STRATEGIES.items():
    metrics = evaluate_strategy(strategy['allocations'])
    simulation_results.append({'dalys': metrics['total_dalys'], 'q1_share': metrics['q1_share'], 'name': name})

sim_df = pd.DataFrame(simulation_results)

# Plot
fig = go.Figure()
unnamed = sim_df[sim_df['name'].isna()] if 'name' in sim_df.columns else sim_df
fig.add_trace(go.Scatter(x=unnamed['dalys'], y=unnamed['q1_share'], mode='markers',
    marker=dict(size=5, color='lightgray', opacity=0.5), name='Random allocations'))

for name, color in zip(['Maximum Efficiency', 'Equity Focus', 'Balanced Portfolio'], ['#2ecc71', '#3498db', '#9b59b6']):
    point = sim_df[sim_df.get('name') == name]
    if len(point) > 0:
        fig.add_trace(go.Scatter(x=point['dalys'], y=point['q1_share'], mode='markers+text',
            marker=dict(size=15, color=color), text=[name], textposition='top center', name=name))

fig.add_hline(y=20, line_dash="dash", line_color="red", annotation_text="Equal distribution")
fig.update_layout(title='The Efficiency-Equity Trade-off',
    xaxis_title='Total DALYs (Efficiency →)', yaxis_title='Q1 Share % (Equity →)', height=500)
fig.show()

## 7. Incorporating Uncertainty

In [ ]:
# Evaluate strategies with uncertainty
uncertainty_results = {}
for name, strategy in STRATEGIES.items():
    samples = evaluate_strategy_with_uncertainty(strategy['allocations'], n_samples=1000, seed=42)
    uncertainty_results[name] = {
        'samples': samples, 'mean': np.mean(samples),
        'p5': np.percentile(samples, 5), 'p95': np.percentile(samples, 95)
    }

# Visualise
fig = go.Figure()
colors = ['#2ecc71', '#3498db', '#9b59b6']
for i, (name, results) in enumerate(uncertainty_results.items()):
    fig.add_trace(go.Violin(y=results['samples'], name=name, box_visible=True,
        meanline_visible=True, fillcolor=colors[i], opacity=0.7))

fig.update_layout(title='DALYs Averted with Uncertainty (Monte Carlo)', yaxis_title='DALYs', height=500, showlegend=False)
fig.show()

print("\n90% Confidence Intervals:")
for name, r in uncertainty_results.items():
    print(f"{name}: {r['mean']:,.0f} [{r['p5']:,.0f} - {r['p95']:,.0f}]")

## 8. Equity Weighting

Should a DALY averted in a deprived area count for more?

In [ ]:
# Interactive equity weight explorer
equity_slider = FloatSlider(value=1.0, min=1.0, max=3.0, step=0.1,
    description='Q1 Weight:', continuous_update=True, style={'description_width': 'initial'})
equity_output = Output()

def update_equity_analysis(change):
    q1_weight = equity_slider.value
    weights = {
        'Q1': q1_weight, 'Q2': 1 + (q1_weight - 1) * 0.6, 'Q3': 1.0,
        'Q4': 1 - (q1_weight - 1) * 0.3, 'Q5': 1 - (q1_weight - 1) * 0.5
    }
    
    with equity_output:
        clear_output(wait=True)
        print(f"Weights: Q1={weights['Q1']:.1f}, Q2={weights['Q2']:.1f}, Q3={weights['Q3']:.1f}, Q4={weights['Q4']:.1f}, Q5={weights['Q5']:.1f}")
        
        results = []
        for name, strategy in STRATEGIES.items():
            weighted = evaluate_with_equity_weights(strategy['allocations'], weights)
            unweighted = evaluate_strategy(strategy['allocations'])['total_dalys']
            results.append({'Strategy': name, 'Unweighted': unweighted, 'Weighted': weighted, 'Change': (weighted/unweighted-1)*100})
        
        df = pd.DataFrame(results).sort_values('Weighted', ascending=False)
        display(df.style.format({'Unweighted': '{:,.0f}', 'Weighted': '{:,.0f}', 'Change': '{:+.1f}%'}).hide(axis='index'))

equity_slider.observe(update_equity_analysis, names='value')
print("Explore how equity weighting changes rankings:")
display(equity_slider, equity_output)
update_equity_analysis(None)

## 9. Build Your Own Strategy

In [ ]:
# Define your strategy
my_strategy = {
    'salt_reduction': 0,
    'folic_acid': 0,
    'weight_management': 0,
    'diabetes_prevention': 0,
    'school_meals': 0,
    'smoking_cessation': 0,
    'hypertension_screening': 0,
    'sdil_extension': 0
}

total = sum(my_strategy.values())
print(f"Allocated: £{total}M / £50M")

if total > 0:
    metrics = evaluate_strategy(my_strategy)
    print(f"DALYs averted: {metrics['total_dalys']:,.0f}")
    print(f"Cost per DALY: £{metrics['cost_per_daly']:,.0f}")
    print(f"Q1 share: {metrics['q1_share']:.1f}%")

## 10. Discussion Questions

1. **What drove your choices?** Cost-effectiveness, equity, evidence quality?

2. **Did you fund school meals?** High cost per DALY but strong equity impact. How do you value children's future health?

3. **Should we use equity weights?** Who decides how much more a DALY in Middlesbrough is worth?

4. **What's missing?** Political feasibility, implementation capacity, public preferences...

---

## Key Takeaways

1. Cost-effectiveness varies by scale (diminishing returns)
2. Efficiency and equity often trade off
3. Uncertainty should influence decisions
4. Time horizon shapes priorities
5. Values are embedded in methods

---

## References

- NICE (2013). Guide to the methods of technology appraisal.
- WHO (2017). Tackling NCDs: Best buys.
- Cookson R et al. (2017). Cost-effectiveness and health equity. *Value in Health*.